<h1><center>Лабораторна робота 7.</center></h1>
<h2><center>Аналіз активності людини за відкритими даними мобільних телефонів</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

<a class="anchor" id="lab-7"></a>

## Зміст

- [7.1. Завантаження та підготовка даних](#lab-7.1)
- [7.2. Застосування методу головних компонент](#lab-7.2)  
- [7.3. Використання *k*-середніх](#lab-7.3)
- [7.4. Агломератична кластеризація](#lab-7.4)
- [7.5. Застосування класифікатора після кластеризації](#lab-7.5)
- [7.6. Класифікація із пониженням розмірності](#lab-7.6)

<a class="anchor" id="lab-7.1"></a>

## <span style="color:blue; font-size:1.2em;">7.1. Завантаження та підготовка даних</span>

[Повернутися до змісту](#lab-7)

У цій лабораторній робота Вам потрібно розібратися з тим, як працюють методи пониження розмірності й кластеризації даних. Крім того, ще раз попрактикуємося із задачею класифікації.

Тут ми працюватимемо з набором даних [Human Activity Recognition with Smartphones](https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones). Завантажте дані (чотири файли у форматі .txt) з [репозиторію](https://github.com/radiukpavlo/intelligent-data-analysis/tree/main/02_assignments/ida_lab-07_pca-kmeans/samsung_HAR) на GitHub.

Набір даних розпізнавання людської активності був створена на основі записів 30 учасників дослідження, які виконували повсякденні дії, тримаючи на поясі смартфон з вбудованими інерційними датчиками. Мета — класифікувати активності на одну з шести виконаних дій.

Експерименти проводились із групою з 30 добровольців віком від 19 до 48 років. Кожен учасник виконував шість дій (ХОДЬБА, ХОДЬБА ПО СХОДАХ ВГОРУ, ХОДЬБА ПО СХОДАХ ВНИЗ, СИДІННЯ, СТОЯННЯ, ЛЕЖАННЯ), водночас носив смартфон (Samsung Galaxy S II) на поясі. Отриманий набір даних був випадково розділений на дві частини: 70% учасників було відібрано для формування навчальних даних, а 30% — для тестових даних.

Для кожного запису в наборі даних наведено такі ознак (стовпці):

- Тривісне прискорення від акселерометра (загальне прискорення) та оцінене прискорення тіла.
- Тривісна кутова швидкість від гіроскопа.
- Вектор з 561 ознаки, що містить змінні з часової та частотної областей.
- Маркер активності.
- Ідентифікатор учасника, який виконував експеримент.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

# Встановлення стилю для графіків та параметрів відображення
%matplotlib inline
plt.style.use(['seaborn-darkgrid'])
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

# Випадковий стан для відтворюваності результатів
RANDOM_STATE = 17

C:\Users\radiu\AppData\Local\Temp\ipykernel_29672\3083233972.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-darkgrid'])


In [2]:
# Завантаження даних, якщо код пишеться в редакторі з Anaconda
X_train = np.loadtxt("./samsung_HAR/samsung_train.txt")
y_train = np.loadtxt("./samsung_HAR/samsung_train_labels.txt").astype(int)

X_test = np.loadtxt("./samsung_HAR/samsung_test.txt")
y_test = np.loadtxt("./samsung_HAR/samsung_test_labels.txt").astype(int)

In [3]:
# Перевіримо розмірності
assert(X_train.shape == (7352, 561) and y_train.shape == (7352,))
assert(X_test.shape == (2947, 561) and y_test.shape == (2947,))

Для кластеризації нам не потрібен вектор відповідей, тому будемо працювати з об'єднанням навчальної та тестової вибірок. Об'єднаємо `X_train` з `X_test` та `y_train` з `y_test`.

In [4]:
X = np.vstack([X_train, X_test])
y = np.hstack([y_train, y_test])

Визначимо кількість унікальних значень міток цільового класу.

In [5]:
np.unique(y)

array([1, 2, 3, 4, 5, 6])

In [6]:
n_classes = np.unique(y).size
n_classes

6

Ці мітки [відповідають](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.names) таким значенням:
- 1 – ходьба;
- 2 – підйом вгору сходами;
- 3 – спуску вниз сходами;
- 4 – сидіння;
- 5 – стояння;
- 6 – лежання.

Виконаємо масштабування вибірки за допомогою `StandardScaler` з параметрами за замовчуванням:

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

------------------------

<a class="anchor" id="lab-7.2"></a>

## <span style="color:blue; font-size:1.2em;">7.2. Застосування методу головних компонент</span>

[Повернутися до змісту](#lab-7)

### <span style="color:red; font-size:1.5em;">Завдання 1</span>

---

**Для всіх варіантів:**

**Виконайте пониження розмірності даних за допомогою PCA. Залишіть таку кількість компонент, щоби пояснити як мінімум 90% дисперсії оброблених (масштабованих) даних. Використайте масштабовану вибірку й зафіксуйте random_state (константа RANDOM_STATE).**

---

**Варіант 1:** Виконайте пониження розмірності набору даних `samsung_HAR` за допомогою `PCA`. Залиште достатню кількість компонент, щоби пояснити як мінімум 95 % дисперсії масштабованих даних. Оцініть, як змінюється точність класифікації за допомогою $k$-середніх за даними зі зменшеною розмірністю. Результати порівняйте з оригінальними даними без пониження розмірності.

*Технічна примітка:* Використайте `Pandas` для оброблення даних, `StandardScaler` з `sklearn.preprocessing` – для масштабування, `PCA` з `sklearn.decomposition` – для аналізу компонент, і `KMeans` – для кластеризації.

---

**Варіант 2:** Використайте інкрементний `PCA` (Incremental PCA) для пониження розмірності даних `samsung_HAR`, щоб оптимізувати використання пам’яті. Застосуйте цей метод до великої підмножини даних, а потім порівняйте точність класифікації з результатами стандартного `PCA`. Проаналізуйте, наскільки результативним є інкрементний метод у випадку оброблення значного обсягу даних.

*Технічна примітка:* Використайте `IncrementalPCA` з `sklearn.decomposition` та визначте обсяг даних для оброблення в одному циклі за допомогою параметра `batch_size`. Для масштабування скористайтесь `StandardScaler` із `sklearn.preprocessing`.

---

**Варіант 3:** Виконайте пониження розмірності даних `samsung_HAR` за допомогою ядерного `PCA` (Kernel PCA) з різними ядрами (радіально-базисне RBF, поліноміальне та сигмоїдне). Порівняйте результати кластеризації з використанням кожного з ядер і візуалізуйте отримані кластери. Поясніть, як вибір ядра впливає на результат кластеризації.

*Технічна примітка:* Застосовуйте `KernelPCA` з `sklearn.decomposition`, вибираючи різні параметри `kernel` ('rbf', 'poly', 'sigmoid'). Для кластеризації використайте `KMeans`, а для візуалізації – `Matplotlib`.

---

**Варіант 4:** Використайте метод розрідженого `PCA` (Sparse PCA) для пониження розмірності даних `samsung_HAR`, щоб отримати компоненти, які легше інтерпретувати. Проаналізуйте, як Sparse `PCA` впливає на точність кластеризації проти стандартного `PCA`, та поясніть значення розрідженості для інтерпретації результатів.

*Технічна примітка:* Використайте `SparsePCA` з `sklearn.decomposition`, вказавши бажану кількість компонент та коефіцієнт регуляризації. Для кластеризації використайте `KMeans`, а для візуалізації – `Matplotlib`.

---

**Варіант 5:** Дослідіть вплив різної кількості компонентів `PCA` на якість кластеризації даних `samsung_HAR`. Використайте метрику оцінки кластерів (силует) для аналізу та визначте оптимальну кількість компонентів за методом «ліктя». Результати візуалізуйте.

*Технічна примітка:* Для оцінювання кластерів застосуйте `silhouette_score` з `sklearn.metrics` до отриманих результатів кластеризації. Візуалізація та визначення оптимальної кількості компонентів – за допомогою `Matplotlib`.

---

**Варіант 6:** Порівняйте методи пониження розмірності: стандартний `PCA` та лінійний дискримінантний аналіз (LDA) за даними `samsung_HAR`. Оцініть точність кластеризації обома методами та поясніть різницю між некерованим підходом `PCA` та керованим LDA.

*Технічна примітка:* Використайте `PCA` та `LinearDiscriminantAnalysis` з `sklearn.decomposition`. Для кластеризації використайте `KMeans`, а для оцінювання – `accuracy_score` з `sklearn.metrics`.

---

**Варіант 7:** Проведіть експеримент зі зведенням даних `samsung_HAR` до трьох компонентів за допомогою `PCA`. Візуалізуйте отримані кластери на діаграмі розсіювання та проаналізуйте, наскільки `PCA` допомагає виокремити основні патерни в даних.

*Технічна примітка:* Використайте `PCA` з `sklearn.decomposition` для отримання трьох компонентів, а для візуалізації даних використайте `scatter` з `Matplotlib`.

---

**Варіант 8:** Використайте `PCA` з обертанням Varimax для отримання компонентів, що мають чіткіше розподілені навантаження. Проаналізуйте, як ротація впливає на інтерпретацію результатів та на якість кластеризації в задачі пониження розмірності даних `samsung_HAR`.

*Технічна примітка:* Використайте `FactorAnalysis` з параметром `rotation='varimax'` з `factor_analyzer`. Для кластеризації скористайтеся `KMeans` із `sklearn.cluster`.

---

**Варіант 9:** Поєднайте `PCA` з відбором ознак SelectKBest за даними `samsung_HAR`. Проведіть експерименти з різною кількістю відібраних ознак та кількістю компонентів `PCA`, щоб визначити оптимальний баланс між відбором ознак і зменшенням розмірності для точнішої кластеризації.

*Технічна примітка:* Застосовуйте `SelectKBest` з `sklearn.feature_selection` до вхідних даних для відбору ознак, а `PCA` для зменшення розмірності. Для кластеризації використайте `KMeans`.

---

**Варіант 10:** Застосуйте `PCA` до окремих підмножин даних `samsung_HAR` (розділіть за певними категоріями діяльності) та порівняйте результати кластеризації між однорідними та неоднорідними групами. Оцініть, як пониження розмірності працює з різними типами даних.

*Технічна примітка:* Використайте `Pandas` для поділу даних на підмножини, `PCA` – для пониження розмірності та `KMeans` – для кластеризації.

---

**Варіант 11:** Дослідіть результативність пониження розмірності для набору даних `samsung_HAR` за допомогою `PCA`, використовуючи різні методи попередньої нормалізації, як от `MinMaxScaler` та `StandardScaler`. Порівняйте результати кластеризації після пониження розмірності з обома методами масштабування, обґрунтувавши, як вибір масштабу впливає на якість кластерів.

*Технічна примітка:* Використайте `MinMaxScaler` та `StandardScaler` з `sklearn.preprocessing` – для нормалізації даних, `PCA` – для зменшення розмірності, а для кластеризації – `KMeans`.

---

**Варіант 12:** Виконайте `PCA` за набором `samsung_HAR`, щоб зменшити кількість ознак до 90 % поясненої дисперсії, а потім порівняйте результати кластеризації з використанням двох різних методів кластеризації: `KMeans` та `Agglomerative Clustering`. Поясніть переваги та недоліки кожного методу для цього завдання.

*Технічна примітка:* Використайте `PCA` з `sklearn.decomposition` для зменшення розмірності, `KMeans` та `AgglomerativeClustering` з `sklearn.cluster` – для кластеризації. Для оцінювання кластера Використайте `silhouette_score` з `sklearn.metrics`.

---

**Варіант 13:** Розгляньте вплив вибору кількості компонент `PCA` на класифікацію даних `samsung_HAR` у задачі багатокласової класифікації. Залиште від 2 до 10 компонент та оцініть точність класифікації для кожного варіанту з використанням `RandomForestClassifier`. Порівняйте результати та визначте оптимальну кількість компонент.

*Технічна примітка:* Використайте `PCA` для вибору різної кількості компонент, `RandomForestClassifier` з `sklearn.ensemble` – для класифікації, а `accuracy_score` з `sklearn.metrics` – для оцінювання.

---

**Варіант 14:** Використайте `PCA` для зменшення розмірності даних `samsung_HAR`, потім порівняйте результативність кластеризації за допомогою алгоритмів `KMeans` та `DBSCAN`. Оцініть, як структура кластерів змінюється у випадку переходу від одного алгоритму до іншого.

*Технічна примітка:* Для зменшення розмірності застосуйте `PCA` з `sklearn.decomposition` до вхідних даних, для кластеризації використайте `KMeans` та `DBSCAN` з `sklearn.cluster`, а для оцінювання якості – `silhouette_score`.

---

**Варіант 15:** Дослідіть можливість зниження розмірності даних `samsung_HAR` за допомогою `PCA` до 2D та 3D-простору для візуалізації. Створіть графічні зображення для кожного випадку, порівнявши розташування кластерів у двовимірному та тривимірному просторі, щоб оцінити, наскільки легко розрізнити групи даних.

*Технічна примітка:* Використайте `PCA` для зменшення розмірності, `Matplotlib` – для візуалізації у 2D та `mpl_toolkits.mplot3d` – для 3D візуалізації.

---

**Варіант 16:** Виконайте `PCA` за набором `samsung_HAR`, залишаючи 85 % поясненої дисперсії, та використайте результат як вхідні дані для класифікації за допомогою `SVM`. Проаналізуйте, як зменшення розмірності впливає на точність та швидкість класифікації.

*Технічна примітка:* Використайте `PCA` для пониження розмірності, `SVC` з `sklearn.svm` – для класифікації та `accuracy_score` з `sklearn.metrics` – для оцінювання.

---

**Варіант 17:** Застосуйте метод пониження розмірності `PCA` до вхідних даних `samsung_HAR`, зберігаючи 95 % поясненої дисперсії. Порівняйте результативність класифікації з використанням простої нейронної мережі до та після пониження розмірності. Визначте, чи допомагає `PCA` у швидкості навчання та загальній результативності моделі.

*Технічна примітка:* Використайте `PCA` для зменшення розмірності, `MLPClassifier` з `sklearn.neural_network` – для класифікації та `accuracy_score` – для оцінювання результативності.

---

**Варіант 18:** Виконайте `PCA` для зменшення розмірності даних `samsung_HAR`, зберігаючи лише 80 % дисперсії, а потім кластеризуйте зменшені дані за допомогою `MeanShift`. Проаналізуйте якість кластерів та обґрунтуйте, чому цей метод може бути кращим для виявлення природних груп даних.

*Технічна примітка:* Використайте `PCA` для пониження розмірності, `MeanShift` з `sklearn.cluster` – для кластеризації та `silhouette_score` – для оцінювання якості кластерів.

---

**Варіант 19:** Виконайте `PCA` з обертанням Varimax для набору `samsung_HAR` та збережіть компоненти, що пояснюють не менше 90 % дисперсії. Потім проаналізуйте отримані компоненти та спробуйте зрозуміти, які ознаки активності вони можуть відображати. Поясніть, як обертання впливає на інтерпретованість компонентів.

*Технічна примітка:* Для обертання використайте `FactorAnalysis` з параметром `rotation='varimax'`. Використайте `Pandas` для оброблення даних.

---

**Варіант 20:** Дослідіть використання `PCA` у поєднанні з функцією `SelectFromModel` для зменшення розмірності набору `samsung_HAR`. Використайте `RandomForestClassifier` як модель для відбору важливих ознак перед застосуванням `PCA`. Оцініть точність класифікації після оброблення даних цими методами.

*Технічна примітка:* Використайте `SelectFromModel` з `sklearn.feature_selection`, `RandomForestClassifier` з `sklearn.ensemble` – для відбору ознак, та `PCA` – для пониження розмірності.

------------------------

<a class="anchor" id="lab-7.3"></a>

## <span style="color:blue; font-size:1.2em;">7.3. Використання $k$-середніх</span>

[Повернутися до змісту](#lab-7)

Виконайте кластеризацію даних методом `Kmeans`, навчивши модель за даними зі зниженою за рахунок PCA розмірністю. Тут потрібно шукати саме 6 кластерів, але загалом у задачах без вчителя ми не знаємо, яку кількість кластерів треба шукати.

Параметри:

- **n_clusters** = n_classes (кількість унікальних міток цільового класу);
- **n_init** = 100;
- **random_state** = RANDOM_STATE (для відтворюваності результату).

Інші параметри залишаються за замовчуванням.

### <span style="color:red; font-size:1.5em;">Завдання 2</span>


---

**Варіант 1:** Використайте метрики `Adjusted Rand Index`, `Adjusted Mutual Information` та Silhouette для оцінювання якості кластеризації даних `samsung_HAR` методом $k$-середніх. Порівняйте отримані кластери з істинними мітками, щоб оцінити результативність кластеризації. Обговоріть, чому ці метрики є корисними для оцінювання кластеризації.

*Технічна примітка:* Скористайтесь `adjusted_rand_score`, `adjusted_mutual_info_score` та `silhouette_score` з `sklearn.metrics` для оцінювання кластеризації.

---

**Варіант 2:** Дослідіть, як зміна кількості кластерів впливає на якість кластеризації на основі набору `samsung_HAR`. Використайте метрики Silhouette Score та Davies-Bouldin Index для вибору оптимальної кількості кластерів і проаналізуйте, як це значення змінює результат кластеризації. Дайте оцінку результатів із погляду поділу різних видів людської активності.

*Технічна примітка:* Використайте `KMeans` з параметром `n_clusters`, змінюючи його значення, а для оцінювання якості кластерів використайте `silhouette_score` та `davies_bouldin_score` з `sklearn.metrics`.

---

**Варіант 3:** Порівняйте результативність звичайного `KMeans` та `MiniBatchKMeans` на великій підмножині даних `samsung_HAR`, щоб оцінити переваги `MiniBatchKMeans` щодо збереження пам’яті та часу обчислень. Проаналізуйте, як обидва алгоритми виконують кластеризацію та які їхні обмеження.

*Технічна примітка:* Використайте `KMeans` та `MiniBatchKMeans` з `sklearn.cluster`, а також метрики `inertia_` для вимірювання внутрішньої узгодженості кластерів.

---

**Варіант 4:** Проведіть кластеризацію з використанням `KMeans`, попередньо зменшивши розмірність даних `samsung_HAR` за допомогою tSNE. Порівняйте якість кластеризації з результатами `KMeans`, проведеними на повному наборі даних. Визначте, як зменшення розмірності за допомогою tSNE впливає на розподіл кластерів і чи покращує це чіткість поділу даних.

*Технічна примітка:* Використайте `TSNE` з `sklearn.manifold` для зменшення розмірності, а `KMeans` з `sklearn.cluster` – для кластеризації.

---

**Варіант 5:** Використайте різні метрики відстані в $k$-середніх (евклідова, мангеттенська, чебишева) для аналізу кластеризації на `samsung_HAR`. Порівняйте, як кожна метрика відстані впливає на структуру та якість кластерів, і поясніть, чому певні метрики можуть бути краще пристосовані для цього набору даних.

*Технічна примітка:* Використайте `KMeans` з параметром `metric` із бібліотеки `scipy.spatial.distance`, а для оцінювання використайте `silhouette_score`.

---

**Варіант 6:** Виконайте кластеризацію даних `samsung_HAR` за допомогою `KMeans` для виявлення потенційних аномалій. Визначте, які спостереження перебувають значно віддалено від своїх центрів кластерів, й обґрунтуйте, як ці аномалії можна використати для покращення результатів у кластеризації даних про активність людини.

*Технічна примітка:* Використайте `KMeans` з `sklearn.cluster` та метод відстані від кожної точки до центроїду для ідентифікації аномалій.

---

**Варіант 7:** Реалізуйте зважений $k$-середніх, щоби підкреслити значення певних ознак у даних `samsung_HAR`. Проведіть аналіз того, як зважування певних ознак впливає на результати кластеризації. Це особливо корисно для наборів даних, де деякі ознаки мають більший вплив на класифікацію.

*Технічна примітка:* Використайте модифікований `KMeans`, що підтримує зважування ознак. Це можна реалізувати через бібліотеки `Numpy` та `sklearn.cluster`.

---

**Варіант 8:** Виконайте експерименти з різними критеріями збіжності в алгоритмі `KMeans` за даними `samsung_HAR`. Порівняйте результати кластеризації під час зміни критерію за кількістю ітерацій та величиною зміни `inertia`. Обґрунтуйте важливість вибору оптимальних критеріїв для точності та ефективності кластеризації.

*Технічна примітка:* Використайте параметри `max_iter` і `tol` з `KMeans`, а для оцінювання результативності скористайтеся `inertia_` та часом обчислень.

---

**Варіант 9:** Виконайте кластеризацію за набором `samsung_HAR`, використовуючи різні методи ініціалізації $k$-середніх: `k-means++` та `random`. Порівняйте результати кластеризації за допомогою метрики Silhouette Score, щоб оцінити вплив методу ініціалізації на точність та стабільність кластерів. Оцініть швидкість збіжності алгоритму за кожної ініціалізації, пояснивши, які переваги дає кожен метод для набору даних про людську активність.

*Технічна примітка:* Використайте `KMeans` з параметрами `init='k-means++'` та `init='random'` з `sklearn.cluster`, а для оцінювання якості – `silhouette_score` з `sklearn.metrics`.

---

**Варіант 10:** Застосуйте метод $k$-середніх до даних `samsung_HAR`, попередньо перетворивши ознаки за допомогою Box-Cox та Yeo-Johnson. Порівняйте, як кожне перетворення впливає на результати кластеризації, та поясніть, чому вибір перетворення є важливим для цього набору даних.

*Технічна примітка:* Використайте `PowerTransformer` з параметрами `method='box-cox'` та `method='yeo-johnson'` з `sklearn.preprocessing`, а для кластеризації – `KMeans`.

–

**Варіант 11:** Використайте метод $k$-середніх для розподілу даних `samsung_HAR` на кластери, але попередньо розділіть набір даних на підмножини за категоріями людської активності. Порівняйте результати кластеризації для кожної окремої підмножини та для всього набору даних, щоб визначити, чи спостерігаються певні особливості кластеризації для кожного виду активності.

*Технічна примітка:* Використайте `Pandas` для розділення набору на підмножини, `KMeans` з `sklearn.cluster` для кластеризації, а для оцінювання якості кластерів – `silhouette_score`.

---

**Варіант 12:** Дослідіть вплив різних розмірів вибірок на результати кластеризації з використанням $k$-середніх. Виконайте кластеризацію на вибірках різного розміру, взятих із `samsung_HAR`, і проаналізуйте, як розмір вибірки впливає на якість кластерів та стабільність результатів.

*Технічна примітка:* Використайте метод `sample` з `Pandas` для створення різних вибірок, `KMeans` з `sklearn.cluster` для кластеризації, та `silhouette_score` – для оцінювання якості кластерів.

---

**Варіант 13:** Виконайте кластеризацію $k$-середніх за даними `samsung_HAR`, але попередньо проведіть зменшення розмірності даних, використовуючи `PCA`. Порівняйте результати кластеризації зі стандартною кластеризацією без пониження розмірності та обговоріть вплив `PCA` на якість кластерів.

*Технічна примітка:* Використайте `PCA` з `sklearn.decomposition` для зменшення розмірності, `KMeans` для кластеризації, а для оцінювання якості кластерів – `silhouette_score`.

---

**Варіант 14:** Застосуйте метод $k$-середніх до нормалізованих даних `samsung_HAR` і порівняйте результати кластеризації з кластеризацією ненормалізованих даних. Проаналізуйте, як нормалізація впливає на структуру та якість кластерів, і поясніть важливість цього етапу для роботи з різнорідними даними.

*Технічна примітка:* Використайте `StandardScaler` з `sklearn.preprocessing` для нормалізації даних, `KMeans` – для кластеризації, а `silhouette_score` – для оцінювання якості кластерів.

---

**Варіант 15:** Виконайте кластеризацію $k$-середніх за даними `samsung_HAR`, використовуючи різні значення початкових параметрів для `random_state`. Порівняйте результати кластеризації, щоб оцінити вплив `random_state` на стабільність та узгодженість кластерів, особливо для набору даних про людську активність.

*Технічна примітка:* Використайте параметр `random_state` з `KMeans` у `sklearn.cluster`, тестуючи різні значення, а для оцінювання стабільності кластерів застосовуйте `silhouette_score`.

---

**Варіант 16:** Використайте метод $k$-середніх для кластеризації даних `samsung_HAR` та реалізуйте стратегію ініціалізації центрів через ручний вибір початкових центрів на основі попереднього аналізу даних. Порівняйте результати з автоматичними методами ініціалізації та обґрунтуйте вибір ручної стратегії.

*Технічна примітка:* Для вибору початкових центрів Використайте `Pandas` або `Numpy`, а для кластеризації – `KMeans` з параметром `init`.

---

**Варіант 17:** Проведіть кластеризацію $k$-середніх для даних `samsung_HAR`, але Використайте метрику `cosine` для обчислення відстаней між точками. Проаналізуйте, як заміна евклідової метрики на косинусну впливає на структуру та розподіл кластерів.

*Технічна примітка:* Використайте бібліотеку `scipy.spatial.distance` для обчислення косинусної відстані, а для кластеризації – `KMeans` з відповідними змінами в налаштуваннях.

---

**Варіант 18:** Дослідіть метод `KMeans` для виявлення структурних особливостей різних груп даних `samsung_HAR`. Використайте крос-валідацію, щоб оцінити стабільність та якість кластерів для різних розділів вибірки, і визначте, які види діяльності найкраще розділяються за кластерами.

*Технічна примітка:* Використайте `KMeans` з `sklearn.cluster`, `cross_val_score` для крос-валідації та `silhouette_score` – для оцінювання кластерів.

---

**Варіант 19:** Застосуйте метод $k$-середніх до даних `samsung_HAR`, але спробуйте різні варіанти кількості ітерацій у налаштуваннях `max_iter`. Порівняйте результати кластеризації у випадку зміни `max_iter`, щоб оцінити вплив цього параметра на результативність та точність кластеризації для даних про людську активність.

*Технічна примітка:* Використайте параметр `max_iter` з `KMeans` у `sklearn.cluster`, а для оцінювання якості кластерів – `silhouette_score` та час обчислень.

---

**Варіант 20:** Виконайте кластеризацію $k$-середніх за даними `samsung_HAR`, використовуючи адаптивний вибір кількості кластерів на основі розподілу інерції. Використайте метод «лікоть» для визначення оптимальної кількості кластерів і порівняйте результати з фіксованою кількістю кластерів.

*Технічна примітка:* Використайте `KMeans` з `sklearn.cluster`, побудуйте графік інерції за допомогою `Matplotlib`, та оцініть оптимальну кількість кластерів.

------------------------

<a class="anchor" id="lab-7.4"></a>

## <span style="color:blue; font-size:1.2em;">7.4. Агломератична кластеризація</span>

[Повернутися до змісту](#lab-7)

Спробуємо ще один алгоритм, що входить до групи ієрархічної кластеризації – агломеративну кластеризацію [`AgglomerativeClustering()`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html).

### <span style="color:red; font-size:1.5em;">Завдання 3</span>


---

**Варіант 1:** Використайте Adjusted Rand Index (ARI) для порівняння якості кластеризації, виконаної за допомогою агломеративної кластеризації та `MiniBatchKMeans` на основі даних `samsung_HAR`. Проаналізуйте, як обидва методи виділяють групи людської активності та визначте, який метод є результативнішим.

*Технічна примітка:* Використайте `AgglomerativeClustering` та `MiniBatchKMeans` з `sklearn.cluster`, а для оцінювання – `adjusted_rand_score` з `sklearn.metrics`.

---

**Варіант 2:** Виконайте нормалізацію даних `samsung_HAR` з використанням різних методів (StandardScaler, MinMaxScaler, RobustScaler), а потім використайте агломеративну кластеризацію. Порівняйте результати кластеризації для кожного методу нормалізації та обґрунтуйте важливість вибору правильного методу масштабування для аналізу людської активності.

*Технічна примітка:* Використайте `StandardScaler`, `MinMaxScaler` та `RobustScaler` з `sklearn.preprocessing` для нормалізації, а `AgglomerativeClustering` – для кластеризації.

---

**Варіант 3:** Оцініть якість кластерів, отриманих за допомогою агломеративної кластеризації, використовуючи різні індекси валідності, як от Silhouette Score, Calinski-Harabasz, та Davies-Bouldin Index. Поясніть переваги кожного із цих індексів для оцінювання кластеризації на основі даних про людську активність.

*Технічна примітка:* Використайте `silhouette_score`, `calinski_harabasz_score` та `davies_bouldin_score` з `sklearn.metrics` – для оцінювання якості кластерів.

---

**Варіант 4:** Виконайте агломеративну кластеризацію за даними `samsung_HAR`, експериментуючи з різними метриками відстані (евклідова, мангеттенська, чебишева). Порівняйте результати для кожної метрики та обґрунтуйте вибір найкращої метрики відстані для даних про людську активність.

*Технічна примітка:* Використайте `AgglomerativeClustering` з параметром `affinity` для вибору метрики, а для аналізу результатів використайте `silhouette_score` з `sklearn.metrics`.

---

**Варіант 5:** Використайте різні значення параметра `distance_threshold` в Агломеративній кластеризації за даними `samsung_HAR` для визначення оптимальної кількості кластерів. Дослідіть, як зміна порогу відстані впливає на кількість і структуру кластерів та на відповідність отриманих груп реальним типам активності.

*Технічна примітка:* Використайте `AgglomerativeClustering` з параметром `distance_threshold`, залишаючи `n_clusters=None`. Використайте `silhouette_score` для оцінювання якості кластерів за різних значень порогу.

---

**Варіант 6:** Застосуйте метод агломеративної кластеризації до вхідних даних `samsung_HAR` з різними кількостями ознак після зменшення розмірності за допомогою `PCA`. Дослідіть, як зміна кількості головних компонент впливає на результат кластеризації та структуру кластерів.

*Технічна примітка:* Використайте `PCA` з `sklearn.decomposition` для вибору кількості компонент, а потім використайте `AgglomerativeClustering`. Оцінка за `silhouette_score` для порівняння результатів.

---

**Варіант 7:** Проведіть агломеративну кластеризацію за нормалізованими та ненормалізованими даними `samsung_HAR`. Порівняйте якість кластеризації та визначте, чи є нормалізація необхідною для підвищення точності розподілу на кластери. Зробіть висновки про важливість нормалізації для такого типу даних.

*Технічна примітка:* Використайте `StandardScaler` з `sklearn.preprocessing` для нормалізації даних та `AgglomerativeClustering` для кластеризації. Порівняйте результати за `silhouette_score`.

---

**Варіант 8:** Застосуйте агломеративну кластеризацію до даних `samsung_HAR`, використовуючи різні критерії зв’язку (Ward, complete, average, single) та порівняйте отримані результати. Визначте, який критерій найбільш вдало відокремлює різні види людської активності, проаналізувавши сформовану структуру кластерів. Зробіть висновки про придатність кожного критерію для цього набору даних.

*Технічна примітка:* Використайте `AgglomerativeClustering` з `sklearn.cluster` із параметром `linkage` для вибору критеріїв зв’язку. Для оцінювання результатів кластеризації використайте `silhouette_score` з `sklearn.metrics`.

---

**Варіант 9:** Виконайте агломеративну кластеризацію за даними `samsung_HAR`, попередньо зменшивши їхню розмірність за допомогою `PCA` і t-SNE. Проаналізуйте вплив зменшення розмірності на якість кластеризації та структуру кластерів, порівнявши результати обох методів. Обґрунтуйте вибір найкращого підходу для даного набору даних.

*Технічна примітка:* Використайте `PCA` та `TSNE` з `sklearn.decomposition` та `sklearn.manifold` для зменшення розмірності, а потім застосуйте `AgglomerativeClustering` до результатів кластеризації.

---

**Варіант 10:** Реалізуйте метод перехресної перевірки для оцінювання стабільності кластерів, отриманих за допомогою агломеративної кластеризації за даними `samsung_HAR`. Дослідіть, як стабільність кластерів змінюється за різних параметрів моделі, і зробіть висновки про якість та надійність кластеризації в неконтрольованому середовищі.

*Технічна примітка:* Використайте `AgglomerativeClustering` з `sklearn.cluster` та реалізуйте крос-валідацію, розбиваючи дані за допомогою `KFold` з `sklearn.model_selection`.

---

**Варіант 11:** Використайте агломеративну кластеризацію з обмеженням зв’язності на основі $k$-найближчих сусідів. Проаналізуйте, як додавання зв’язності впливає на форму, щільність та розмір кластерів для даних `samsung_HAR`. Поясніть, чому цей підхід може бути корисним для набору даних про людську активність.

*Технічна примітка:* Застосуйте `AgglomerativeClustering` до вхідних даних із параметром `connectivity`, створеним за допомогою `kneighbors_graph` із `sklearn.neighbors`.

---

**Варіант 12:** Побудуйте дендрограму для ієрархічної структури кластерів, отриманої з агломеративної кластеризації. Використовуючи дендрограму, визначте оптимальну кількість кластерів та проаналізуйте, як ієрархічна структура відображає подібність різних видів людської активності.

*Технічна примітка:* Використайте `dendrogram` з бібліотеки `scipy.cluster.hierarchy` для побудови ієрархічної структури на основі результатів `AgglomerativeClustering`.

---

**Варіант 13:** Використайте результати кластерів, сформованих за допомогою агломеративної кластеризації, як додаткові ознаки в моделі класифікації (наприклад, `RandomForestClassifier`). Оцініть, як використання кластерних ознак впливає на результативність моделі та чи можуть ці ознаки покращити точність у задачах розпізнавання людської активності.

*Технічна примітка:* Використайте `AgglomerativeClustering` для формування кластерів, а `RandomForestClassifier` з `sklearn.ensemble` – для оцінювання результативності моделі класифікації.

---

**Варіант 14:** Використайте різні значення параметра `n_clusters` в Агломеративній кластеризації на основі даних `samsung_HAR`, щоб дослідити оптимальну кількість кластерів. Застосуйте метод «лікоть» для визначення оптимальної кількості кластерів та обґрунтуйте ваш вибір для цього набору даних.

*Технічна примітка:* Використайте `AgglomerativeClustering` з `n_clusters`, застосовуючи `silhouette_score` для оцінювання якості кластеризації та графік «лікоть» для вибору оптимальної кількості.

---

**Варіант 15:** Застосуйте метод агломеративної кластеризації до різних підмножин даних `samsung_HAR`, розділених за певними категоріями активності. Проаналізуйте, чи відрізняється структура кластерів для різних підмножин та зробіть висновки про вплив окремих видів активності на кластеризацію.

*Технічна примітка:* Використайте `Pandas` для створення підмножин даних, `AgglomerativeClustering` – для кластеризації та `silhouette_score` – для порівняння результатів.

---

**Варіант 16:** Виконайте агломеративну кластеризацію на основі підмножин, сформованих із даних `samsung_HAR`, що належать до різних груп людської активності, та створіть дендрограми для кожної підмножини. Порівняйте дендрограми та визначте, чи спільні особливості активності можна виявити в кожній групі.

*Технічна примітка:* Використайте `dendrogram` із `scipy.cluster.hierarchy` для побудови дендрограм та `AgglomerativeClustering` – для кластеризації.

---

**Варіант 17:** Використайте різні значення параметра `affinity` в Агломеративній кластеризації, щоб експериментувати з різними метриками відстані для даних `samsung_HAR`. Проаналізуйте, як кожна метрика впливає на результати кластеризації, та виберіть оптимальну для даного набору даних.

*Технічна примітка:* Застосовуйте `AgglomerativeClustering` із параметром `affinity` (`euclidean`, `manhattan`) для вибору метрики відстані та `silhouette_score` – для порівняння результатів.

---

**Варіант 18:** Застосуйте агломеративну кластеризацію з обмеженнями на основі часової зв’язності для даних `samsung_HAR`, щоб врахувати послідовність подій. Дослідіть, як включення такої інформації впливає на розподіл кластерів для видів активності, що відбуваються в часовій послідовності.

*Технічна примітка:* Використайте `AgglomerativeClustering` з обмеженням `connectivity` на основі `kneighbors_graph`, з огляду на послідовні часові етапи.

---

**Варіант 19:** Виконайте агломеративну кластеризацію на основі даних `samsung_HAR`, створюючи кластери для певних значень груп активності. Порівняйте результати кластеризації для кожного виду активності та визначте, чи краще їх класифікувати окремо або як спільну групу.

*Технічна примітка:* Використайте `AgglomerativeClustering` для кластеризації кожної активності окремо та `silhouette_score` – для оцінювання якості кластеризації.

---

**Варіант 20:** Виконайте агломеративну кластеризацію з різними значеннями параметра `linkage` (наприклад, Ward, complete, average) для даних `samsung_HAR`. Оцініть вплив різних підходів до злиття на структуру кластерів і поясніть, який підхід краще підходить для аналізу цього набору даних.

*Технічна примітка:* Використайте параметр `linkage` із `AgglomerativeClustering` та порівняйте якість кластерів за допомогою `silhouette_score`.

------------------------

<a class="anchor" id="lab-7.5"></a>

## <span style="color:blue; font-size:1.2em;">7.5. Застосування класифікатора після кластеризації</span>

[Повернутися до змісту](#lab-7)

Варто відзначити, що попереднє завдання не надто добре розв'язується саме як задача кластеризації, якщо виділяти лише кілька кластерів (>2). Спробуємо тепер розв'язати задачу класифікації; пам'ятаємо, що дані у нас розмічені.

### <span style="color:red; font-size:1.5em;">Завдання 4</span>


---

**Варіант 1:** Проведіть зменшення розмірності даних за допомогою `PCA` та t-SNE перед класифікацією, потім порівняйте результативність класифікаторів за різними метриками. Оцініть, як зменшення розмірності різними методами впливає на точність класифікації кластеризованих даних.

*Технічна примітка:* Використайте `PCA` та `tSNE` для зменшення розмірності, `LinearSVC` – для класифікації та `classification_report` – для оцінювання результатів.

---

**Варіант 2:** Реалізуйте стековий ансамбль із різних базових класифікаторів, застосованих після кластеризації даних `samsung_HAR`. Порівняйте результативність стекової моделі з індивідуальними моделями, обґрунтуйте, як синергія декількох класифікаторів покращує результат.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `StackingClassifier` з `sklearn.ensemble`, а для оцінювання – `classification_report`.

---

**Варіант 3:** Застосуйте метод ансамблювання Gradient Boosting до кластеризованих даних за `KMeans` і порівняйте їхню результативність із простішими моделями, як от дерево рішень та логістична регресія. Проаналізуйте, як кластеризація покращує роботу ансамблевих моделей для розпізнавання активності.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `GradientBoostingClassifier` – для ансамблювання, а `classification_report` – для оцінювання результативності.

---

**Варіант 4:** Додайте до початкових ознак нову ознаку, що відображає відстань кожного спостереження до центроїда свого кластера. Потім застосуйте класифікатор `RandomForestClassifier` і оцініть, як ця нова ознака впливає на точність класифікації видів активності. Порівняйте результати з моделлю без додаткової ознаки.

*Технічна примітка:* Використайте `KMeans` для кластеризації та обчислення відстаней до центроїдів, `RandomForestClassifier` – для класифікації, `classification_report` – для оцінювання результатів.

---

**Варіант 5:** Після кластеризації застосуйте `PCA` до кластерних міток, щоб зменшити їхню розмірність, а потім використайте ці компоненти як нові ознаки для класифікації. Оцініть, як використання зменшених кластерних ознак впливає на точність класифікації в задачі розпізнавання активності.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `PCA` з `sklearn.decomposition` – для зменшення розмірності та `SVC` – для класифікації.

---

**Варіант 6:** Використайте кластеризацію для групування схожих ознак у наборі даних `samsung_HAR`, а потім застосуйте класифікацію до об’єднаних груп ознак. Порівняйте результативність моделі до й після об’єднання ознак і визначте, чи підвищує це точність класифікації.

*Технічна примітка:* Використайте `KMeans` для кластеризації ознак, `RandomForestClassifier` – для класифікації, `classification_report` – для оцінювання результатів.

---

**Варіант 7:** Створіть нову ознаку для кожного кластера, яка відображає середнє значення ознак у межах кластера, та використайте її для класифікації видів активності. Оцініть, як ця ознака впливає на класифікацію, порівнявши результати з початковою моделлю.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `Pandas` – для обрахування середніх значень, а для класифікації – `LogisticRegression` з `sklearn.linear_model`.

---

**Варіант 8:** Використайте кластеризацію для створення нової ознаки, яка відображає різницю між спостереженням та центроїдом свого кластера. Потім застосуйте класифікатор `GradientBoostingClassifier` до нових даних і оцініть, як ця різниця впливає на результативність класифікації.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `GradientBoostingClassifier` з `sklearn.ensemble` – для класифікації, `classification_report` – для оцінювання результативності.

---

**Варіант 9:** Порівняйте результативність різних класифікаторів (SVM, Random Forest, Logistic Regression) після кластеризації даних `samsung_HAR`. Виконайте кластеризацію як етап попереднього оброблення, а потім оцініть, як вона впливає на точність і стабільність кожного класифікатора. Поясніть, чому певні моделі можуть краще взаємодіяти з кластеризованими даними.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `SVC`, `RandomForestClassifier`, і `LogisticRegression` з `sklearn` – для класифікації та `classification_report` – для оцінювання.

---

**Варіант 10:** Застосуйте випадковий ліс та метод беггінгу до кластеризованих даних для оцінювання важливості ознак у класифікаційній задачі. Оцініть, як кластеризація змінює значення ознак і впливає на загальну результативність моделей. Поясніть роль різних ознак у виявленні людської активності.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `RandomForestClassifier` та `BaggingClassifier` з `sklearn.ensemble`, а також `feature_importances_` для аналізу важливості ознак.

---

**Варіант 11:** Використайте класифікатор `LinearSVC` з `GridSearchCV` для оптимізації гіперпараметрів після попередньої кластеризації даних. Проаналізуйте, як кластеризація впливає на налаштування оптимальних параметрів та на точність класифікації. Визначте, який вид активності класифікатор визначає найгірше за метрикою $F_1$-score.

*Технічна примітка:* Використайте `GridSearchCV` з `LinearSVC` та `KMeans`, а для масштабування – `StandardScaler` з `sklearn.preprocessing`.

---

**Варіант 12:** Застосуйте різні ансамблеві методи (Random Forest, AdaBoost) до кластеризованих даних і порівняйте їх за метриками accuracy, precision, recall, $F_1$-score та ROC-AUC. Оцініть, як кластеризація впливає на ці метрики та як ансамблі можуть покращити розпізнавання людської активності.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `RandomForestClassifier` та `AdaBoostClassifier` з `sklearn.ensemble`, а для оцінювання – `classification_report`.

---

**Варіант 13:** Порівняйте результативність `LinearSVC` після кластеризації за допомогою різних алгоритмів: $k$-середніх, агломеративної кластеризації та DBSCAN. Використайте `classification_report` для оцінювання результативності та визначте, який алгоритм кластеризації найкраще розділяє види активності для покращення результатів класифікації.

*Технічна примітка:* Використайте `KMeans`, `AgglomerativeClustering`, та `DBSCAN` з `sklearn.cluster` для кластеризації, а `LinearSVC` – для класифікації.

---

**Варіант 14:** Проведіть кластеризацію даних `samsung_HAR`, а потім застосуйте модель `AdaBoostClassifier` до кластеризованих даних. Порівняйте результати з класифікацією на основі оригінальних даних без кластеризації, щоб оцінити, як кластеризація впливає на точність ансамблевих методів.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `AdaBoostClassifier` з `sklearn.ensemble` – для класифікації, `classification_report` – для оцінювання результатів.

---

**Варіант 15:** Застосуйте методи кластеризації з обмеженням на зв’язність, щоб групувати ознаки на основі їхньої просторової близькості, а потім використайте ці кластери як ознаки для класифікації. Оцініть, як кластеризація на основі зв’язності впливає на точність класифікації.

*Технічна примітка:* Використайте `AgglomerativeClustering` з параметром `connectivity`, `RandomForestClassifier` для класифікації та `classification_report` – для оцінювання результатів.

---

**Варіант 16:** Застосуйте `StackingClassifier` з різними базовими класифікаторами (SVM, Random Forest, KNN) до даних після їхньої кластеризації. Оцініть результативність стекового ансамблю та порівняйте його результати з продуктивністю поодиноких моделей.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `StackingClassifier` з `sklearn.ensemble`, `classification_report` – для оцінювання.

---

**Варіант 17:** Виконайте кластеризацію за даними `samsung_HAR`, а потім застосуйте метод `VotingClassifier` до кластеризованих даних із різними моделями для їхньої класифікації. Порівняйте результативність голосування з одиничними класифікаторами та оцініть синергію кластеризації та ансамблевого голосування.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `VotingClassifier` з `sklearn.ensemble`, `classification_report` – для оцінювання результативності.

---

**Варіант 18:** Дослідіть вплив багаторазової кластеризації з різними параметрами (кількість кластерів) на результативність класифікації. Для кожного варіанта кластеризації додайте відповідну мітку до даних, а потім використайте `LogisticRegression` для класифікації, щоб оцінити, як різні кластери впливають на точність моделі.

*Технічна примітка:* Використайте `KMeans` для створення декількох варіантів кластеризації, `LogisticRegression` – для класифікації та `classification_report` – для оцінювання точності.

---

**Варіант 19:** Застосуйте `LinearSVC` до кластеризованих даних за $k$-середніх та налаштуйте його гіперпараметри з використанням `GridSearchCV`. Порівняйте результативність моделі з та без кластеризації. Проаналізуйте, як кластеризація впливає на оптимальні гіперпараметри та точність у задачах розпізнавання активності.

*Технічна примітка:* Використайте `KMeans` для кластеризації, `LinearSVC` – для класифікації, `GridSearchCV` – для налаштування параметрів, а також `classification_report` – для оцінювання результатів.

---

**Варіант 20:** Розгляньте мітки кластерів як нові ознаки та додайте їх до основного набору даних. Потім застосуйте `LinearSVC` для класифікації та обрахуйте результативність моделі. Проаналізуйте, як додаткові ознаки впливають на точність та результативність класифікатора для різних видів людської активності.

*Технічна примітка:* Використайте `KMeans` для створення міток кластерів, `LinearSVC` – для класифікації, `classification_report` – для оцінювання за метриками.

------------------------

<a class="anchor" id="lab-7.6"></a>

## <span style="color:blue; font-size:1.2em;">7.6. Класифікація із пониженням розмірності</span>

[Повернутися до змісту](#lab-7)

### <span style="color:red; font-size:1.5em;">Завдання 5</span>


---

**Варіант 1:** Застосуйте `PCA` до набору даних `samsung_HAR` для зменшення розмірності та об’єднайте отримані результати з `t-SNE` для створення двовимірного візуального подання даних перед класифікацією. Виконайте класифікацію з використанням `RandomForestClassifier` та проаналізуйте, як `PCA` і t-SNE разом можуть покращити візуалізацію та розуміння класифікаційних результатів. Оцініть результативність класифікатора за метриками: accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `PCA` для зменшення розмірності, `TSNE` з `sklearn.manifold` – для візуалізації та `RandomForestClassifier` – для класифікації, `classification_report` з `sklearn.metrics` – для оцінювання.

---

**Варіант 2:** Проведіть поетапну кластеризацію з `MiniBatchKMeans` після зменшення розмірності за `PCA`, щоб зменшити обчислювальні витрати на великих даних. Оцініть результативність кластеризації та точність подальшої класифікації з використанням `LogisticRegression`. Проаналізуйте метрики класифікації: accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `MiniBatchKMeans` з `sklearn.cluster`, `PCA` для зменшення розмірності та `LogisticRegression` – для класифікації. Оцініть результативність за допомогою `classification_report`.

---

**Варіант 3:** Використайте метод `NMF` (негативно-матричний факторизаційний аналіз) для зменшення розмірності даних `samsung_HAR`, а потім класифікуйте за допомогою `RandomForestClassifier`. Порівняйте результативність NMF з `PCA` для розпізнавання видів активності. Оцініть модель за accuracy, precision, recall та $F_1$-score.

*Технічна примітка:* Використайте `NMF` з `sklearn.decomposition` та `RandomForestClassifier` для класифікації. Порівняйте результати за допомогою `classification_report`.

---

**Варіант 4:** Застосуйте метод поетапного відбору ознак перед `PCA` (`SequentialFeatureSelector`), щоб виділити найбільш релевантні ознаки, а потім класифікуйте дані за допомогою `SVC`. Оцініть вплив комбінованого підходу на результативність класифікатора, використовуючи метрики: accuracy, precision, recall та $F_1$-score.

*Технічна примітка:* Використайте `SequentialFeatureSelector` з `sklearn.feature_selection`, `PCA` для зменшення розмірності та `SVC` – для класифікації.

---

**Варіант 5:** Застосуйте `LDA` як метод зменшення розмірності перед класифікацією за допомогою $k$-найближчих сусідів. Проаналізуйте, як LDA впливає на точність класифікації та результативність моделі проти результатів `PCA`, оцінюючи accuracy, precision, recall і $F_1$-score.

*Технічна примітка:* Використайте `LinearDiscriminantAnalysis` з `sklearn.discriminant_analysis` для зменшення розмірності та `KNeighborsClassifier` з `sklearn.neighbors`.

---

**Варіант 6:** Після застосування `PCA` до набору даних `samsung_HAR` створіть ансамбль моделей (`RandomForestClassifier`, `SVC`, `MLPClassifier`) й об’єднайте їхні результати через метод голосування (`VotingClassifier`). Оцініть, як `PCA` та голосування впливають на точність та результативність класифікації видів активності за метриками accuracy, precision, recall та $F_1$-score.

*Технічна примітка:* Використайте `PCA` для зменшення розмірності, `VotingClassifier` з `sklearn.ensemble`, а для класифікації – `RandomForestClassifier`, `SVC`, `MLPClassifier`.

---

**Варіант 7:** Використайте метод `SparsePCA` для зменшення розмірності даних `samsung_HAR` та застосуйте класифікатор `LogisticRegression`. Оцініть, як розрідженість у Sparse `PCA` впливає на точність та стабільність моделі, порівнюючи результати з традиційним `PCA`. Оцініть модель за accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `SparsePCA` з `sklearn.decomposition`, `LogisticRegression` з `sklearn.linear_model`, а для оцінювання – `classification_report`.

---

**Варіант 8:** Комбінуйте `FactorAnalysis` з класифікатором `GradientBoostingClassifier`, щоби проаналізувати, як факторний аналіз у порівнянні з `PCA` впливає на точність класифікації. Зробіть висновки про застосування `Factor Analysis` для виявлення латентних ознак у даних про людську активність, використовуючи метрики accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `FactorAnalysis` з `sklearn.decomposition`, `GradientBoostingClassifier` з `sklearn.ensemble` та `classification_report` – для оцінювання результативності.

---

**Варіант 9:** Застосуйте методи кластеризації `KMeans` перед `PCA`, щоб сформувати кластери, а потім зменште розмірність кожного кластера окремо. Після цього використайте модель `SVC` для класифікації на основі об’єднаних результатів кластеризації та `PCA`. Оцініть точність класифікації за допомогою метрик accuracy, precision, recall та $F_1$-score.

*Технічна примітка:* Використайте `KMeans` з `sklearn.cluster` для кластеризації, `PCA` – для зменшення розмірності та `SVC` – для класифікації.

---

**Варіант 10:** Використайте `IncrementalPCA` для поступового зменшення розмірності даних `samsung_HAR`, щоб зменшити витрати пам’яті для великого обсягу даних. Після пониження розмірності застосуйте класифікатор `RandomForestClassifier` та порівняйте точність класифікації з результатами стандартного `PCA`, аналізуючи метрики accuracy, precision, recall та $F_1$-score.

*Технічна примітка:* Використайте `IncrementalPCA` з `sklearn.decomposition` для поетапного зменшення розмірності, `RandomForestClassifier` – для класифікації та `classification_report` – для оцінювання.

---

**Варіант 11:** Застосуйте стековий класифікатор (`StackingClassifier`), де кожен базовий класифікатор навчається на різних рівнях зменшення розмірності, отриманих із `PCA`. Оцініть, як об’єднання моделей, навчених за різними рівнями розмірності, впливає на загальну результативність у задачі класифікації, використовуючи метрики accuracy, precision, recall і $F_1$-score.

*Технічна примітка:* Використайте `PCA` для зменшення розмірності, `StackingClassifier` з `sklearn.ensemble`, та `classification_report` – для оцінювання результатів.

---

**Варіант 12:** Використайте метод `TruncatedSVD` замість `PCA` для зменшення розмірності, а потім скористайтесь логістичною регресією з регуляризацією L1 та L2. Оцініть, як вибір методу зменшення розмірності та типу регуляризації впливає на точність класифікації, аналізуючи метрики accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `TruncatedSVD` з `sklearn.decomposition`, `LogisticRegression` з параметрами `penalty='l1'` та `penalty='l2'`.

---

**Варіант 13:** Виконайте масштабування даних за допомогою `StandardScaler`, `MinMaxScaler` та `RobustScaler` перед пониженням розмірності з `PCA` та класифікацією з `SVC`. Оцініть вплив кожного методу масштабування на точність класифікації з використанням моделі `SVC`, аналізуючи accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `StandardScaler`, `MinMaxScaler` та `RobustScaler` з `sklearn.preprocessing` для масштабування, `PCA` – для зменшення розмірності та `SVC` – для класифікації.

---

**Варіант 14:** Застосуйте метод `PCA`, а потім класифікатор $k$-найближчих сусідів із різними метриками відстані (евклідова, мангеттенська, чебишева). Проаналізуйте, як `PCA` впливає на точність класифікації залежно від метрики відстані в класифікаторі KNN. Оцініть модель за accuracy, precision, recall та $F_1$-score.

*Технічна примітка:* Використайте `PCA` для зменшення розмірності, `KNeighborsClassifier` з параметром `metric` з `sklearn.neighbors`.

---

**Варіант 15:** Комбінуйте `PCA` з методами вирівнювання дисбалансу класів, такими як `SMOTE` та `class_weight=balanced`. Оцініть, як ця комбінація впливає на результативність класифікатора, наприклад, `RandomForestClassifier`, за допомогою метрик accuracy, precision, recall, $F_1$-score, та проаналізуйте виклики, що пов’язані з обробленням дисбалансів у просторі зі зменшеною розмірністю.

*Технічна примітка:* Використайте `PCA`, `SMOTE` з `imblearn.over_sampling`, `RandomForestClassifier` з параметром `class_weight` у `sklearn.ensemble`.

---

**Варіант 16:** Скористайтесь `PCA` перед використанням нелінійних класифікаторів, як от `MLPClassifier` та `k-nearest neighbors`, й оцініть їхню результативність за зменшеним набором ознак. Проаналізуйте, як `PCA` впливає на результативність нелінійних моделей та вибір оптимальних налаштувань для кожного класифікатора, використовуючи метрики accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `PCA`, `MLPClassifier` з `sklearn.neural_network` та `KNeighborsClassifier` з `sklearn.neighbors` для класифікації, а `classification_report` – для оцінювання результатів.

---

**Варіант 17:** Зберігайте різні рівні дисперсії (наприклад, 85 %, 90 %, 95 %) під час виконання пониження розмірності набору даних `samsung_HAR` за `PCA` та проаналізуйте, як цей вибір впливає на результативність класифікації з використанням моделі `LogisticRegression`, оцінюючи accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `PCA` з параметром `n_components` з `sklearn.decomposition` для налаштування рівня дисперсії та `LogisticRegression` – для класифікації.

---

**Варіант 18:** Використайте `KernelPCA` з різними ядрами (RBF, поліноміальне, сигмоїдне) у комбінації з класифікатором `SVC`, налаштувавши гіперпараметри за допомогою `GridSearchCV`. Проаналізуйте, як вибір ядра в ядерному `PCA` (Kernel PCA) впливає на результативність класифікації та оптимальні гіперпараметри моделі, оцінюючи accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `KernelPCA` з параметром `kernel` з `sklearn.decomposition`, `SVC` з `sklearn.svm` та `GridSearchCV` з `sklearn.model_selection`.

---

**Варіант 19:** Проведіть експерименти з пониженням розмірності набору даних `samsung_HAR` за `PCA` перед класифікацією на основі ансамблів `AdaBoost` та `GradientBoostingClassifier`. Порівняйте результати з початковими даними без `PCA`, щоб оцінити, як зменшення розмірності впливає на якість ансамблевих моделей у задачі розпізнавання людської активності, використовуючи метрики accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `PCA` для зменшення розмірності, `AdaBoostClassifier` та `GradientBoostingClassifier` з `sklearn.ensemble` – для класифікації, а `classification_report` – для оцінювання результативності.

---

**Варіант 20:** Поєднайте метод `PCA` з відбором ознак за допомогою `SelectKBest` та використайте класифікатор `RandomForestClassifier` для побудови моделі. Оцініть, як комбінація `PCA` та відбору ознак впливає на точність класифікації, а також визначте оптимальну кількість компонент `PCA` для досягнення найкращої результативності, оцінюючи метрики accuracy, precision, recall, $F_1$-score.

*Технічна примітка:* Використайте `PCA` з `sklearn.decomposition`, `SelectKBest` з `sklearn.feature_selection` та `RandomForestClassifier` з `sklearn.ensemble`. Оцінка результатів за допомогою `classification_report`.